In [6]:
print("hello worsld")

hello worsld


In [6]:
import cv2
from ultralytics import YOLO
from datetime import datetime, timedelta
import sys

# ==========================================
# CONFIGURATION
# ==========================================
MODEL_PATH = 'yolov8m.pt' 
TARGET_CLASSES = [0] # Hanya Person
CONF_THRESHOLD = 0.3 

# --- TURBO MODE ---
FRAME_SKIP = 30 # Deteksi tiap 30 frame (Hemat Resource)

def process_video(input_path, output_video_path):
    print(f"[INFO] Loading Model {MODEL_PATH}...")
    model = YOLO(MODEL_PATH)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Waktu awal (Sesuaikan dengan video)
    current_time = datetime.strptime("2025-11-07 00:46:00", "%Y-%m-%d %H:%M:%S")

    print(f"\n[INFO] START PROCESSING")
    print(f"      File: {input_path}")
    print(f"      Resolution: {frame_width}x{frame_height}")
    print(f"      Frame Skip: {FRAME_SKIP}")
    print("-" * 50)

    frame_count = 0
    last_boxes = [] 
    last_person_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1
        
        # Update waktu
        current_time += timedelta(seconds=1/fps)
        time_str = current_time.strftime("%H:%M:%S")

        # --- LOGIKA DETEKSI (Hanya tiap 30 frame) ---
        if frame_count % FRAME_SKIP == 0:
            results = model.predict(frame, classes=TARGET_CLASSES, conf=CONF_THRESHOLD, verbose=False)
            last_boxes = []
            last_person_count = len(results[0].boxes)

            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                last_boxes.append((x1, y1, x2, y2, conf))
        
        # --- VISUALISASI ---
        hour = current_time.hour
        is_night = (hour >= 18 or hour < 6)
        
        status_text = "NORMAL"
        status_color = (0, 255, 0)

        if last_person_count > 0:
            if is_night:
                status_text = "ALERT: INTRUDER!"
                status_color = (0, 0, 255)
            else:
                status_text = "ACTIVITY DETECTED"
                status_color = (0, 255, 255)

        # Gambar Kotak Orang
        for (x1, y1, x2, y2, conf) in last_boxes:
            box_color = (0, 0, 255) if is_night else (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
            label = f"Person {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

        # ========================================================
        # DASHBOARD UI (DIPINDAH KE KIRI BAWAH)
        # ========================================================
        # Hitung koordinat berdasarkan tinggi frame (frame_height)
        dash_height = 120
        dash_y_start = frame_height - dash_height # Mulai dari bawah - 120px
        
        # 1. Background Hitam Transparan
        cv2.rectangle(frame, (0, dash_y_start), (350, frame_height), (0, 0, 0), -1)
        
        # 2. Teks Info (Koordinat Y menyesuaikan dash_y_start)
        # Time
        cv2.putText(frame, f"TIME : {time_str}", (15, dash_y_start + 35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Person Count
        cv2.putText(frame, f"PERSON: {last_person_count}", (15, dash_y_start + 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Status
        cv2.putText(frame, f"STATUS: {status_text}", (15, dash_y_start + 105), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)

        # Tulis ke video output
        out.write(frame)
        
        # Print Progress Bar di Terminal
        if frame_count % 100 == 0:
            progress = (frame_count / total_frames) * 100
            sys.stdout.write(f"\r[Progress] {progress:.1f}% Complete ({frame_count}/{total_frames})")
            sys.stdout.flush()

    cap.release()
    out.release()
    print(f"\n\n[INFO] Selesai! Video tersimpan di: {output_video_path}")

if __name__ == "__main__":
    # Ganti path ini sesuai lokasi videomu
    input_video = r"D:\cctv\datalumina-project-template\data\raw\1117.mp4"
    output_video = r"output_security_turbo.mp4"
    
    process_video(input_video, output_video)

[INFO] Loading Model yolov8m.pt...

[INFO] START PROCESSING
      File: D:\cctv\datalumina-project-template\data\raw\1117.mp4
      Resolution: 1280x720
      Frame Skip: 30
--------------------------------------------------
[Progress] 98.3% Complete (1600/1628)

[INFO] Selesai! Video tersimpan di: output_security_turbo.mp4
